In [ ]:
import numpy
from tqdm import tqdm
!pip install Unidecode
import unidecode
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget 'https://github.com/morfologik/polimorfologik/releases/download/2.1/polimorfologik-2.1.zip'
!unzip '/content/polimorfologik-2.1.zip'
!unzip '/content/drive/MyDrive/Colab Notebooks/NLP/Dane/Copy of polish_corpora.zip'
!wget 'http://zil.ipipan.waw.pl/NKJPNGrams?action=AttachFile&do=get&target=2grams.gz'
!gunzip '/content/NKJPNGrams?action=AttachFile&do=get&target=2grams.gz'
!wget 'http://zil.ipipan.waw.pl/NKJPNGrams?action=AttachFile&do=get&target=1grams.gz'
!gunzip '/content/NKJPNGrams?action=AttachFile&do=get&target=1grams.gz'

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/NLP/Dane/Copy of polish_corpora.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/NLP/Dane/Copy of polish_corpora.zip
  inflating: polish_corpora.txt      


# Loading data for uppercase correction 

In [ ]:
BL_uni = set({})

for line in open('/content/polimorfologik-2.1.txt', 'r'):
    word = line.split(';')[1]
    if word[0] == word[0].upper():
        BL_uni.add(word.lower())
    if word[0] == word[0].lower() and word.lower() in BL_uni:
        BL_uni.remove(word.lower())

In [ ]:
BL_bi = set({})

def isalphapolish(word):
    alphabet = 'pchnąćwtęłódźjeżalubośmskrzyńfig'
    for letter in word.lower():
        if letter not in alphabet:
            return False
    return True

for line_num, line in enumerate(open('/content/polish_corpora.txt','r')):
    if line_num == 1000000: break
    line = line.split()
    counter = 1
    while counter < len(line)-1:
        word_1 = line[counter]
        word_2 = line[counter+1]

        if word_1[0] == word_1[0].upper() and word_2[0] == word_2[0].upper():
            BL_bi.add((word_1.lower(),word_2.lower()))

        if (word_1[0] == word_1[0].lower() and word_2[0] == word_2[0].lower() and (word_1.lower(), word_2.lower()) in BL_bi):
            BL_bi.remove((word_1.lower(), word_2.lower()))

        counter += 1

# Loading data for diacritic letter correction

In [ ]:
import unidecode

In [ ]:
words = set({})

for i, line in enumerate(open('/content/polish_corpora.txt', 'r')):
    if i < 1000000: continue
    for word in line.split():
        words.add(unidecode.unidecode(word.lower()))
    if i == 1200000:
        break

In [ ]:
bigrams = {}

for i, line in tqdm(enumerate(open('/content/NKJPNGrams?action=AttachFile&do=get&target=2grams','r'))):
    oc, w1, w2 = line.split()
    w1_de = unidecode.unidecode(w1)
    w2_de = unidecode.unidecode(w2)

    if w1_de in words or w2_de in words:
        key = (w1_de, w2_de)
        if key in bigrams.keys():
            bigrams[key].append(((w1,w2),oc))
        else:
            bigrams[key] = [((w1,w2),oc)]
    
    if i >= 50000000: break
    


49985181it [05:46, 151194.84it/s]

In [ ]:
unigrams = {}

for i, line in tqdm(enumerate(open('/content/NKJPNGrams?action=AttachFile&do=get&target=1grams','r'))):
    oc, w1 = line.split()
    w1_de = unidecode.unidecode(w1)

    if w1_de in words:
        key = w1_de
        if key in unigrams.keys():
            unigrams[key].append((w1,oc))
        else:
            unigrams[key] = [(w1,oc)]

## Uppercase letter correction

In [ ]:
def correct_lowercase_sen_with_bigrams(test):
    i = 0
    test[0] = test[0].capitalize()
    while i < len(test) - 1:

        if (test[i].lower(), test[i+1].lower()) in BL_bi:
            test[i] = test[i].capitalize()
            test[i+1] = test[i+1].capitalize()
        else:
            if test[i].lower() in BL_uni:
                test[i] = test[i].capitalize()
        
        i+=1
    
    if test[i] in BL_uni:
        test[i] = test[i].capitalize()

    return test

# Diacritic letter correction

In [ ]:
def choose_word_softmax(words):

    occurrences = np.array([word[1] for word in words], dtype=np.float64)

    occurrences = occurrences / np.sum(occurrences)

    number_of_words = len(words)
    probs = occurrences
    word_idx = np.random.choice(number_of_words , size = 1, p = probs)[0]

    return words[word_idx][0]

In [ ]:
def correct_diacritic_sen_with_bigrams(test):


    res = []

    if (test[0], test[1]) in bigrams.keys():
        word_1, word_2 = choose_word_softmax(bigrams[(test[0], test[1])])
        res.append(word_1)
        res.append(word_2)
    elif (test[0]) in unigrams.keys():
        word_1 = choose_word_softmax(unigrams[test[0]])
        res.append(word_1)
    else:
        res.append(test[0])

        
    while len(res) <= len(test) - 1:

        last_cor = res[-1]
        bi_scd = test[len(res)] 
        bi_first = test[len(res) - 1]

        if (bi_first, bi_scd) in bigrams.keys():

            words = [((bigram[0], bigram[1]), oc) for bigram, oc in bigrams[(bi_first, bi_scd)] if bigram[0] == last_cor]
            if len(words) == 0:
                if (bi_scd) in unigrams.keys():
                    word_1 = choose_word_softmax(unigrams[bi_scd])
                    res.append(word_1)
            else:
                word_1, word_2 = choose_word_softmax(words)
                res.append(word_2)

        elif (bi_scd) in unigrams.keys():
            word_1 = choose_word_softmax(unigrams[bi_scd])
            res.append(word_1)

        else:
            res.append(test[len(res)])
    
    return res


# TEST

In [ ]:
def acc(target, label):
    res = 0

    for w_label, w_target in zip(label, target):
        if w_label == w_target:
            res += 1
            
    return res/len(label)

In [ ]:
res_1 = 0
res_2 = 0

counter = 0
for i, line in enumerate(open('/content/polish_corpora.txt', 'r')):
    if i < 1000000: continue

    label_diacritic = line.lower().split()
    label_full = line.split()

    target = unidecode.unidecode(line).lower().split()

    cor_dia = correct_diacritic_sen_with_bigrams(target)
    res_1 += acc(cor_dia, label_diacritic)

    cor_full = correct_lowercase_sen_with_bigrams(cor_dia)

    res_2 += acc(cor_full, label_full)
    counter += 1

    if i == 1200000:
        break

In [ ]:
res_1/counter

0.978992170823962

In [ ]:
res_2/counter

0.933766306780607

In [ ]:
((res_1/counter)*(res_2/counter))**(1/2)

0.9561118677840058